In [1]:
from pyspark.sql import *

In [2]:
# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

In [3]:
Employee = Row("firstName", "lastName", "email", "salary")

In [4]:
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)

In [5]:
#Connect Department with Employee
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4])
departmentWithEmployees3 = Row(department=department3, employees=[employee1, employee4])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])

In [6]:
df1 = spark.createDataFrame([departmentWithEmployees1,departmentWithEmployees2])

In [7]:
display(df1)

In [8]:
df2 = spark.createDataFrame([departmentWithEmployees3,departmentWithEmployees4])

In [9]:
unionDF = df1.union(df2)

In [10]:
display(unionDF)

In [11]:
dbutils.fs.rm('/tmp/data/df.parquet',True)

In [12]:
unionDF.write.parquet('/tmp/data/df.parquet')


In [13]:
parquetDF = spark.read.parquet('/tmp/data/df.parquet')

In [14]:
display(parquetDF)

In [15]:
eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a":"b"})])
eDFx = spark.createDataFrame([Row(a=1,  mapfield={"a":"b"}, intlist=[1,2,3])])

In [16]:
display(eDFx)

In [17]:
from pyspark.sql.functions import explode

display(eDF.select('a',explode(eDF.intlist).alias("anInt")))

In [18]:
eDF.select(['intlist']).collect()

In [19]:
#eDF.createGlobalTempView("xyz")
display(spark.sql("select * from xyz"))

In [20]:
display(eDF.select(explode(eDF.mapfield).alias("k","v")))

In [21]:
display(parquetDF)

In [22]:
df = parquetDF.select("department",explode("employees").alias("e"))

In [23]:
display(df)

In [24]:
df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
display(df)

In [25]:
from pyspark.sql.functions import concat
d = df.select(concat(df.s, df.d).alias('s'))


In [26]:
from pyspark.sql.functions import abs

In [27]:
df = spark.createDataFrame([('abcd',-123)], ['s', 'd'])
d = df.select(concat(df.s, abs(df.d)).alias('s'))
display(d)

In [28]:
explodeDF = df.selectExpr("e.firstName","e.lastName","e.email","e.salary")

In [29]:
display(explodeDF)

In [30]:
filterDF = explodeDF.filter(explodeDF.firstName == "michael")

In [31]:
display(filterDF)

In [32]:
display(explodeDF.sort(explodeDF.salary))

In [33]:
display(explodeDF)

In [34]:
filterDF = explodeDF.sort(explodeDF.salary)

In [35]:
from pyspark.sql.functions import col,asc

In [36]:
explodeDF.filter((col("firstName") == "xiangrui") | (col("firstName") == "michael" )).collect()

In [37]:
explodeDF.fillna("--").collect()

In [38]:
explodeDF.filter(col("firstName").isNull()).collect()

In [39]:
from pyspark.sql.functions import countDistinct,count

In [40]:
countDistinctDF = explodeDF.select("firstName","lastName").groupby("firstName").agg(count("firstName"))

In [41]:
display(countDistinctDF)

In [42]:
from pyspark.sql.window import Window